In [1]:
import pandas as pd
import numpy as np

In [2]:
data= pd.read_csv('../MA_PREDICTOR/data/ma_data_car.csv', parse_dates=['announcement_date'])

In [3]:
data

,id,announcement_date,deal_value,acquiror_total_assets,acquiror_name,acquiror_ticker,target_name,target_nation,acquiror_nation,target_status,...,shares_acquired,consideration_offered,attitude,acquisition_technique,acquiror_financial_advisor,target_financial_advisor,purpose,acquiror_code,target_code,car
0,1631732020,2005-01-01,12.00,3160.41,Plains All American Pipeline LP,PAA,Shell Pipeline Co LP-Crude Oil Pipeline Assets...,United States,United States,Subsidiary,...,100.0,Cash|Cash Only,Friendly,Financial Acquiror|Divestiture,NaN,NaN,Strengthen existing operations/expand presence...,50102030,50103030,0.006854
1,1653652020,2005-01-01,NaN,1680.57,Regis Corp,RGS,Scot Lewis Schools,United States,United States,Private,...,100.0,Unspecified,Friendly,Not Applicable,NaN,NaN,NaN,54201030,63103010,-0.010266
2,1949421020,2005-01-01,NaN,3301.76,Google Inc,GOOG,PhatBits,United States,United States,Private,...,100.0,Unspecified,Friendly,Not Applicable,NaN,NaN,NaN,57201030,57201020,0.007746
3,1652453040,2005-01-01,20.03,371.71,Graco Inc,GGG,Gusmer Europe SL,Spain,United States,Private,...,100.0,Cash|Cash Only,Friendly,Not Applicable,NaN,NaN,NaN,52102010,51101010,-0.011133
4,2721963020,2005-01-01,18220.52,NaN,Exxon Mobil Corp,XOM,Exxon Mobil Corp,United States,United States,Public,...,5.0,Cash|Cash Only,No Applicable,Open Market Purchase|Privately Negotiated Purc...,NaN,NaN,Other,50102030,50102030,-0.003971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18454,3791733020,2021-08-30,40.00,2830.00,Chart Industries Inc,GTLS,AdEdge Holdings LLC,United States,United States,Private,...,100.0,Cash Only|Cash,Friendly,Not Applicable,NaN,Raymond James,NaN,52102010,59103010,0.020108
18455,3791806020,2021-08-30,NaN,2224.71,Patrick Industries Inc,PATK,Tumacs Boat Covers Inc,United States,United States,Private,...,100.0,Unspecified,Friendly,Not Applicable,NaN,NaN,NaN,53203020,53205020,-0.040156
18456,3793728040,2021-08-30,NaN,21061.02,Sysco Corp,SYY,Cegal AS,Norway,United States,Subsidiary,...,100.0,Unspecified,Friendly,Divestiture,NaN,NaN,NaN,54301020,57201010,-0.003545
18457,3792444020,2021-08-31,NaN,5295.10,Financial Institutions Inc,FISI,North Woods Capital Benefits LLC,United States,United States,Private,...,100.0,Unspecified,Friendly,Not Applicable,NaN,NaN,NaN,55101010,52203030,-0.025992
